<a href="https://colab.research.google.com/github/uteyechea/crime-prediction-using-artificial-intelligence/blob/master/validate_rnn_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import gc
import sys

import pandas as pd
from scipy import stats

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path='/content/drive/My Drive/Colab Notebooks/crime_prediction'

sys.path.append(path+'/libs')
import temporal_correlation as tc


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [70]:
prediction_file_path=os.path.join(path,'data','prediction','series_sample.txt')
prediction_file=pd.read_csv(prediction_file_path,sep=',')
print(prediction_file.shape)

(2555, 1)


In [71]:
prediction_file.iloc[:,0] = pd.to_numeric(prediction_file.iloc[:,0], errors='coerce') #Remove non float values, substitute them with NAN

In [72]:
prediction_file=prediction_file.dropna()

In [73]:
prediction_file.shape

(2527, 1)

In [74]:
prediction_file.isnull().values.any()

False

In [4]:
training_data_path=os.path.join(path,'data','training','theft2.csv')
training_file=pd.read_csv(training_data_path,sep=',')

In [5]:
training_file.isnull().values.any()

False

In [ ]:
type(prediction_file)

pandas.core.frame.DataFrame

In [ ]:
type(training_file)

pandas.core.frame.DataFrame

In [42]:
prediction_file.shape

(2555, 1)

In [ ]:
training_file.shape

(892, 1)

In [75]:
prediction_file

,-0.0.791945932798457
0,0.466208
1,-0.791946
2,0.969469
4,0.969469
5,-0.037054
...,...
2550,-0.540315
2551,0.717838
2552,0.466208
2553,0.717838


In [38]:
def series_window(dataframe,end,periods,column):
  if type(column)==str:
    column=dataframe.columns.get_loc(column)
  return dataframe.iloc[end-periods:end,column]

def series_correlation(series1,series2):
  #assert something?
  series1=series1.reset_index(drop=True) #Better find a way to simplify this procedure
  series2=series2.reset_index(drop=True) #Better find a way to simplify this procedure
  ro=series1.corr(series2) 
  return ro 

def correlated_series_timestamp(dataframe1,dataframe2,periods,column,min_correlation):
  timestamps=[]
  series1=series_window(dataframe1,periods,periods,column)
  for epoch in reversed(range(2*periods,len(dataframe2))):
    series2=series_window(dataframe2,epoch-periods,periods,column)
    try:
      correlation=series_correlation(series1,series2)
      #print(correlation)
      if correlation >= min_correlation:
        #record datetime value. We will use this datetime value to generate all sequences that will go as input to the RNN
        #print(dataframe.index[epoch]) 
        timestamps.append(dataframe2.index[epoch])
    except:
      print('Can not compute correlation at index ',str(epoch))    
  return timestamps


In [84]:
timestamps=correlated_series_timestamp(training_file,prediction_file,periods=10,column=0,min_correlation=0.85)

In [85]:
timestamps

[2365, 2347, 226]

In [48]:
series1=series_window(prediction_file,end=10,periods=10,column=0)

In [49]:
series2=series_window(training_file,end=10,periods=10,column=0)

In [52]:
series_correlation(series1,series2)

TypeError: ignored

In [56]:
for i,element in enumerate(prediction_file):
  print(type(prediction_file.iloc[i,0]),element)

<class 'str'> -0.0.791945932798457
